# ELYZA-tasks-100 による性能評価

In [ ]:
!pip install accelerate datasets jsonlines sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
!pip install hf_transfer
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/deepseek-llm-67b-chat-GGUF \
    --local-dir . \
    --local-dir-use-symlinks False \
    --include='*Q4_K_M*gguf*'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 37.2 MB/s eta 0:00:00
downloading https://huggingface.co/TheBloke/deepseek-llm-67b-chat-GGUF/resolve/bab8a570ac9271b502108dc7f80eae3881df430a/deepseek-llm-67b-chat.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmpxz9bcj2x
deepseek-llm-67b-chat.Q4_K_M.gguf: 100% 40.4G/40.4G [02:17<00:00, 294MB/s]
/content


## 🚬Smoke test

`llama.cpp` は、デフォルトではメモリマップされたファイルを使用する。`--no-mmap` を有効化する。

[llama.cpp](https://github.com/ggerganov/llama.cpp/issues/2589#issuecomment-1693312217) を参照。

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.20-cp310-cp310-manylinux_2_35_x86_64.whl size=7116166 sha256=50a6be987339dfc596d49ed8ca79bb304128a0af7512a0bc56e20ab5275007dc
  Stored in directory: /root/.cache/pip/wheels/ef/f2/d2/0becb03047a348d7bd9a5b91ec88f4654d6fa7d67ea4e84d43
Successfully built llama-cpp-python


In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="./deepseek-llm-67b-chat.Q4_K_M.gguf",
    n_gpu_layers=90,
    use_mmap=False,
    seed=42,
    n_ctx=2048,
    n_threads=11,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 1 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
# Simple inference example
# Question: https://www.reddit.com/r/LocalLLaMA/comments/13zz8y5/what_questions_do_you_ask_llms_to_check_their/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button
prompt = """User: Is it morally right to kill mosquitoes?

Assistant: """
output = llm(
  prompt, # Prompt
  temperature=0.7,
  echo=False,
  stop=["<｜end▁of▁sentence｜>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  repeat_penalty=1.1,
  seed=42,
  max_tokens=-1,  # Generate up to 512 tokens
)
print(output["choices"][0]["text"])

这个问题涉及到伦理和道德的讨论，不同的人可能会有不同的看法。一些人可能会认为杀死害虫如蚊子是为了保护人类免受疾病的侵害是合理的，因为它们能够传播疾病，例如疟疾、登革热等。然而，也有人认为生命都是宝贵的，即使是小昆虫也应该被尊重和保护。因此，是否杀灭蚊子的决定应该基于个人价值观和对环境的影响考虑。


## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### プロンプトについて

モデルに回答を生成させるときのプロンプトは、各モデルのモデルカード (🤗HuggingFace の README ページ) に記載されているプロンプトを使用する。

> 📒 **ノート**
>
> この方法は、質問を直接入力する場合や、全モデルで共通のプロンプトを用いる場合に比べて公平性が劣る。
> しかし、実利用時は各モデルが最もよい性能を発揮するように、各モデルのプロンプトをチューニングすることが想定される。
> モデルカード記載のプロンプトをチューニング済みのプロンプトと見なすことで、実利用時の性能を比較することを狙いとする。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import jsonlines

def generate_text_with_llama(input_text):
    # Create a chat completion using the Llama object
    prompt = f"""User: {input_text}

Assistant: """
    output = llm(
        prompt,
        temperature=0.7,
        echo=False,
        stop=["<｜end▁of▁sentence｜>"],
        repeat_penalty=1.1,
        seed=42,
        max_tokens=-1,
    )
    return output["choices"][0]["text"]

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text_with_llama(prompt)

            print(f"==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print(f"")

            writer.write({"pred": generated_text})

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)


Llama.generate: prefix-match hit


Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 1. **目標設定**: 新しい目標または挑戦に直面することで、仕事に取り組む意欲を高めることができます。短期的な目標から長期の目標まで、リアルなミニプロジェクトを設定し、達成した時に自分自身に褒美を与えることができます。
2. **変化求め**: 日常的な仕事から逸脱するためには、新しいスキルを学ぶことや、別の役割に移動したり、チーム内で異なるプロジェクトに参加するなどの変化が必要かもしれません。
3. **健康を重視**: 精神的に疲弊している場合、体力的な活動が役立つことがあります。定期的に運動することで、仕事上の熱意を取り戻すためのストレス解消に役立てることができます。
4. **フィードバック受け入れ**: 周囲からのフィードバックを大切視し、改善点を見い出して成長することができます。上司やチームメイトからの建設的な批判に耳を傾けることで、自分自身の仕事に対する熱意を取り戻すことができます。
5. **リラックス**: 過度に仕事に集中すると、逆効果を及ぼす場合もあるため、適切なリラックス時間を確保し、興味があることに取り組むことで精神的な充実感を得ることができます。



Llama.generate: prefix-match hit


Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. クマは、海辺でアザラシと友達になりました。彼女は自由自在な生活が好きで、日々の日常生活から逃れるために海を見ています。

一日、クマは海岸線上に座って、波音と鳥の歌を聴くことに没頭しました。突然、彼女はある動物が泳いでいるのを見ました。アザラシ！クマは驚いた。彼女は海辺に行って初めて、野生の動物と出会うことはなかったのですが、今回は運が良かったようでした。

クマはアザラシを追いかけました。彼女は素早く泳いで、海辺に沿って進みます。最終的に、彼女はアザラシの近くに着いたところで、彼女が友達になりました！

クマは、アザラシと一緒に泳いで楽しんだ日々を送ります。彼女たちは波の中で戯れ合って、潮干から見物しました。クマは心の底から感謝することが出来ます。

一方、クマの家に帰る日々が迫りました。彼女はアザラシを離れることを考え始めました。彼女たちが一緒に過ごす時間は非常に貴重で、クマは心の中で感謝することが出来ます。

最終的に、クマは家を帰る日が来ました。彼女はアザラシに別れの挨拶をし、海辺から離れていきました。ただし、彼女は心の中で永遠に忘れないことが出来ることを誓います。クマは、アザラシと友達になり、自由自在な時間を送ることができたことに感謝することが出来ました。



Llama.generate: prefix-match hit


...

Q. 漢字に対応する意味を選び、対応する数字とアルファベットのペアを解答してください。

1. 立派　A. すばらしい
2. 開拓　B. ある事柄について知っている内容。
3. 知識　C. 物事に味わいや価値を加える
4. 賞賛　D. 新しい領域を切り開く
5. 味付け　E. 高い評価を与える
A. 1. A. すばらしい
2. D. 新しい領域を切り開く
3. B. ある事柄について知っている内容。
4. E. 高い評価を与える
5. C. 物事に味わいや価値を加える



Llama.generate: prefix-match hit


Q. 次の物語の展開を予想してみましょう。

ある日、主人公のもとに不思議な手紙が届きました。手紙には「今夜の満月に、森の奥深くにある洞窟に来てください。あなたを待っています。」と書かれていました。主人公はその手紙に従い、夜中に洞窟の入り口にたどり着きました。中に入ると、謎めいた人物と出会い……。
A. 主人公が洞窟の奥深くに進む途中で、光を受け取ることはなくなり、周囲は暗闇で覆われました。しかし、彼女（彼）は手紙に書かれた通り、洞窟の奥にいる人物を見つけ出すことが出来ました。

そこにいるのは、森の中で住む妖精族の一人でした。彼女（彼）は主人公を待っていたことが判明し、手紙を送ったのも彼女（彼）なのです。

妖精から聞くと、彼女（彼）は森の中で起きる不審な出来事に取り組んでいました。主人公が手助けをすることを求めていたのです。

妖精族は森の守護者で、自然から得る力を使って不調和な出来事に対処する役割がありました。しかし、最近何か変化が起こり、彼女（彼）たちは森の中で住む動物や植物達を守る上で困難に直面しました。

主人公と妖精族の協力で、不調和な出来事が解決されます。森の中で暮らす者たちは安心し、主人公は新たな絆を築いていきます。



Llama.generate: prefix-match hit


Q. 次の文は1つの物語の文の順番をシャッフルしたものです。適切な順に並び替えてください。

1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」
2. 老人は、広い畑で色々な果物を育てていました。
3. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
4. 昔、あるところに老人と孫がいました。
5. 老人は微笑みながら次のように答えました。

A. 1. 「果物だけではなく、人生も同じです。協力しなくては、全てをこなすことはできません。」
2. 昔、あるところに老人と孫がいました。
3. 老人は広い畑で色々な果物を育てていました。
4. ある日、孫が老人に『こんなに広い畑で、全ての果物を一人で育てるのは大変そうですね』と尋ねました。
5. 老人は微笑みながら次のように答えました。

